# 10 - Matching

## What is Regression Doing After All?

지금까지 살펴본 것처럼 회귀 분석은 테스트와 대조군 비교를 할 때 추가 변수를 통제하는 데 놀라운 기능을 발휘합니다. `treatment`와 `outcome`이 서로 독립이면, $(Y_0, Y_1)\perp T | X$, 선형회귀는 추가변수 `X`를 통제하여, `ATE`를 얻을 수 있습니다. 선형회귀가 작동하는 방법은 마법과 같습니다. 직관적인 이해를 위해 모든 추가변수 `X`가 더미변수인 경우를 생각해보겠습니다. 선형회귀는 데이터를 추가변수 `X`값에 따라 더미셀로 분할하고, 실험군과 대조군 평균 차이를 계산합니다. 이 평균 차이는 X 더미의 고정된 셀에서 수행하기 때문에 `X`를 일정하게 유지합니다. 마치 다음과 같이 $E[Y|T=1] - E[Y|T=0] | X=x$ ($x$는 더미셀이며, 예를 들어 모든 추가변수 x=1 인 경우)를 수행하는 것과 같습니다. 그런 다음 선형회귀는 각 셀에서 추정치를 결합하여(각 셀에 대한 `treatment`의 분산과 비례하는 가중치를 적용) `ATE`를 구합니다.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from matplotlib import style
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf

import graphviz as gr

%matplotlib inline

style.use("fivethirtyeight")

예를 들어 질병에 대한 약의 효과를 조사하는 문제를 생각해봅시다. `outcome`은 입원기간입니다. 약의 효과로 입원기간을 줄어들길 기대합니다. 입원 내역을 취합해 남성 6명 여성 4명의 데이터를 얻었습니다. 남성의 참된 인과관계는 -3으로, 약은 입원기간을 3일 줄여줍니다. 여성에 대한 효과는 -2입니다. 남성은 이 질병에 취약해 평균적으로 많이, 오래 입원하는 경향이 있습니다. 또한, 약 처방도 더 많이 받는 편입니다. 전체 6명 남성 중 5명이 약 처방을 받았습니다. 반면, 전체 4명 중 2명의 여성이 약 처방을 받습니다.

In [2]:
drug_example = pd.DataFrame(dict(
    sex= ["M","M","M","M","M","M", "W","W","W","W"],
    drug=[1,1,1,1,1,0,  1,0,1,0],
    days=[5,5,5,5,5,8,  2,4,2,4]
))

실험군과 대조군의 단순한 비교는 음의 방향으로 편향됩니다. 즉, 약은 실제보다 효과가 떨어져 보입니다. `sex`가 `confounder`로 작용하기 때문입니다. 남성은 질병에 취약하므로 입원기간이 길어 추정된 `ATE`는 실제보다 작습니다.

In [3]:
drug_example.query("drug==1")["days"].mean() - drug_example.query("drug==0")["days"].mean()

-1.1904761904761898

남성에 대한 효과는 -3, 여성에 대한 효과는 -2이므로 실제 `ATE`는 -2.6이 되어야 합니다.

$
ATE=\dfrac{(-3*6) + (-2*4)}{10}=-2.6
$

-2.6은 1) 데이터를 `confounder`셀 별로 분할(여기에서는 남성과 여성), 2) 각 셀 별로 효과를 추정, 3) 각 추정치를 가중 평균(가중치는 각 셀 또는 공변량 그룹의 표본 크기)하여 얻을 수 있습니다. 남성과 여성의 표본 크기가 같다면, `ATE` 추정치는 두 그룹의 `ATE`의 중간인 -2.5가 됩니다. 남성 데이터가 더 많으므로 남성의 `ATE`인 -3에 더 가깝습니다. 이러한 추정 방법을 `non-parametric estimate`(비모수 추정)이라 부르는데, 데이터를 만드는 방법에 대한 가정을 하지 않기 때문입니다.

선형회귀로 `sex`를 통제하고자 한다면 선형성 가정을 추가해야 합니다. 선형회귀는 데이터를 남성과 여성으로 나누고 두 그룹에 대한 효과를 추정합니다. 지금까지는 좋습니다. 하지만 선형회귀에서 각 그룹의 효과를 합칠 때 가중치로 표본 크기를 사용하지 않습니다. 대신 그룹 내 `treatment` 분산에 비례하는 가중치를 사용합니다. 예시에서는 남성 1명만 대조군에 속하므로 `treatment` 분산은 남성 그룹이 더 작습니다. 정확히는 남성의 $Var(T)$는 $1/6*(1 - 1/6)=0.139$, 여성의 $Var(T)$는 $2/4*(1 - 2/4)=0.25$입니다. 

In [4]:
print(f"Male Var.: {np.var([1,1,1,1,1,0])}")
print(f"Female Var.: {np.var([1,0,1,0])}")

Male Var.: 0.1388888888888889
Female Var.: 0.25


선형회귀 결과는 $Var(T)$가 높은 여성 그룹에 더 높은 가중치가 적용되어 추정치는 여성의 `ATE`인 -2에 더 가깝습니다.

In [5]:
smf.ols('days ~ drug + C(sex)', data=drug_example).fit().summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,7.5455,0.188,40.093,0.000,7.100,7.990
C(sex)[T.W],-3.3182,0.176,-18.849,0.000,-3.734,-2.902
drug,-2.4545,0.188,-13.042,0.000,-2.900,-2.010


선형회귀는 더미변수와 마찬가지 방법으로 연속변수에 대해 `ATE`를 계산합니다. 공변량 `X`의 분산이 더 큰 방향으로 결과가 산출됩니다.

지금까지 선형회귀의 특이한 방식을 확인했습니다. 선형성이 높고 모수적이며 분산이 높은 `feature`를 좋아합니다. 이러한 특징은 문제에 따라 더 좋을 수도 나쁠 수도 있습니다. 사용할 수 있는 기술 폭을 넓히기 위해 다양한 방법을 익히는 것은 많은 도움이 됩니다. 시도할 방법이 늘어날 뿐 아니라 문제 이해의 폭을 넓혀주기 때문입니다. 여기에서는 `confounder`를 통제하는 **Subclassification Estimator**를 소개합니다.

## The Subclassification Estimator

![img](./data/img/matching/explain.png)

`treatment`에 대한 인과효과를 추정할 때 `confounder`에 주의해야 합니다. 직업훈련 예시처럼 동기가 충분한 사람이 직업훈련을 받는 경향이 있으므로 직업훈련에 상관없이 수입이 더 높을 가능성이 있기 때문입니다. 따라서 동기 부여 수준이 유사한 소규모 그룹에서 직업훈련 효과를 추정해야 합니다.

변수 `X`에 교란요소가 있으면 `X`가 같은 작은 그룹 안에서 실험군과 대조군을 비교해야 합니다. 조건부 독립성 $(Y_0, Y_1)\perp T | X$이 있다면 `ATE`는 아래와 같습니다.

$
ATE = \int(E[Y|X, T=1] - E[Y|X, T=0])dP(x)
$

적분의 역할은 features `X` 분포의 모든 공간을 살펴보고, 모든 작은 공간의 평균의 차이를 계산한다음, 계산된 모든것을 결합하여 `ATE`를 산출하는 것입니다. 불연속적인 `feature` 집합을 생각하면 이해가 쉽습니다. feature `X`가 `K`개의 서로 다른 셀 $\{X_1, X_2, ..., X_k\}$로 이루어져 있다고 말할 수 있습니다. 그리고, 우리가 하는 일은 각 셀의 treatment effect를 계산하고, 이를 결합하여 `ATE`를 구하는 것입니다. 이 불연속적인 경우 적분을 합으로 바꾸면 `subclassifications estimator`가 됩니다.

$
\hat{ATE} = \sum^K_{i=0}(\bar{Y}_{k1} - \bar{Y}_{k0}) * \dfrac{N_k}{N}
$

막대 표기는 평균을 의미하며 $N_{k}$는 $X_k$의 개수입니다. 수식은 각 $X_k$에 대해 국소적인 `ATE`를 계산하여 표본 크기를 가중치로 가중 평균하는 것을 의미합니다. 위의 약 효과 예시에서 -2.6은 `subclassifications estimator`의 결과입니다.

## Matching Estimator

![img](./data/img/matching/its-a-match.png)

`subclassification estimator`는 실제로 많이 사용하지 않습니다. (앞으로 배울 차원의 저주 때문입니다.) 하지만 인과추론이 무엇을 해야 하는지, 교란을 어떻게 제어하는지에 대한 직관을 제공합니다. 10장의 주제인 `Matching Estimator`는 `subclassification estimator`의 확장이라 할 수 있습니다.

아이디어는 매우 비슷합니다. `confounder`에 의해 실험군과 대조군을 비교할 수 없으므로 **유사한 데이터끼리 매칭**합니다. 마치 쌍둥이를 찾는것과 같습니다.

구체적으로 직업훈련이 소득에 미치는 영향을 추정해보겠습니다. 아래는 훈련을 마친 사람들에 대한 데이터입니다.

In [6]:
trainee = pd.read_csv("./data/trainees.csv")
trainee.query("trainees==1")

,unit,trainees,age,earnings
0,1,1,28,17700
1,2,1,34,10200
2,3,1,29,14400
3,4,1,25,20800
4,5,1,29,6100
5,6,1,23,28600
6,7,1,33,21900
7,8,1,27,28800
8,9,1,31,20300
9,10,1,26,28100


다음은 비훈련자들에 대한 데이터입니다.

In [7]:
trainee.query("trainees==0")

,unit,trainees,age,earnings
19,20,0,43,20900
20,21,0,50,31000
21,22,0,30,21000
22,23,0,27,9300
23,24,0,54,41100
24,25,0,48,29800
25,26,0,39,42000
26,27,0,28,8800
27,28,0,24,25500
28,29,0,33,15500


단순하게 평균을 비교하면 훈련을 받은 사람들이 소득이 더 낮습니다.

In [8]:
trainee.query("trainees==1")["earnings"].mean() - trainee.query("trainees==0")["earnings"].mean()

-4297.49373433584

훈련을 받는 사람의 나이가 훨씬 적기 때문입니다. 따라서 나이는 `confounder`입니다. 나이로 매칭하여 문제를 해결해 보겠습니다. 나이가 28로 같은 `unit` 1와 `unit` 27을 짝짓습니다. 마찬가지로 `unit` 2는 34, `unit` 3은 37, `unit` 4는 35와 짝을 이룹니다. `unit` 5는 29살이므로 `unit` 37과 짝지어야 하지만 이미 `unit` 3과 짝이 지어졌습니다. 하지만 같은 `unit`을 여러 번 사용할 수 있습니다. 1개 이상의 `unit`이 일치하면 무작위로 선택합니다.

처음 7개 `unit`에 대해 매칭 결과는 다음과 같습니다.

In [9]:
# make dataset where no one has the same age
unique_on_age = (trainee
                 .query("trainees==0")
                 .drop_duplicates("age"))

matches = (trainee
           .query("trainees==1")
           .merge(unique_on_age, on="age", how="left", suffixes=("_t_1", "_t_0"))
           .assign(t1_minuts_t0 = lambda d: d["earnings_t_1"] - d["earnings_t_0"]))

matches.head(7)

,unit_t_1,trainees_t_1,age,earnings_t_1,unit_t_0,trainees_t_0,earnings_t_0,t1_minuts_t0
0,1,1,28,17700,27,0,8800,8900
1,2,1,34,10200,34,0,24200,-14000
2,3,1,29,14400,37,0,6200,8200
3,4,1,25,20800,35,0,23300,-2500
4,5,1,29,6100,37,0,6200,-100
5,6,1,23,28600,40,0,9500,19100
6,7,1,33,21900,29,0,15500,6400


마지막 `column`인 `t1_minuts_t0`은 소득 차이입니다. `t1_minuts_t0`의 평균은 나이를 통제한 `ATET` 추정치가 됩니다. 앞의 단순한 평균값과 다르게 양수입니다.

In [10]:
matches["t1_minuts_t0"].mean()

2457.8947368421054

예시는 단순히 매칭을 소개하기 위해 만들어진 것입니다. 실제로는 `feature`가 하나 이상이므로 `unit`이 완벽하게 같은 경우는 거의 없습니다. 따라서 `unit`이 얼마나 비슷한지 유사도를 측정해야 합니다. 흔한 지표는 유클리드 거리 $||X_i - X_j||$입니다. 하지만 유클리드 거리는 `feature`의 크기를 반영하지 않습니다. 예를 들어 나이와 같은 특징의 유사도는 나이대와 비교해 10배나 차이 납니다. 값의 단위에 따라 거리가 달라지기 때문입니다. 유클리드 거리에서는 수백 단위의 `feature`에 비해 수 단위의 `feature`는 중요도가 떨어집니다. 따라서 모든 `feature`가 비슷한 범위의 값을 갖도록 크기를 조절해야 합니다.

거리를 정의했으므로 이제 가장 가까운 샘플끼리 짝짓는 것으로 매칭을 수행할 수 있습니다. 수학적으로 `matching estimator`는 아래와 같습니다.

$
\hat{ATE} = \frac{1}{N} \sum^N_{i=0} (2T_i - 1)\big(Y_i - Y_{jm}(i)\big)
$

$Y_{jm}(i)$는 $Y_i$와 가장 유사한 다른 그룹의 `outcome`입니다. $2T_i - 1$는 실험군과 대조군, 두 가지 방법으로 매칭하기 위해 사용합니다. 

`matching estimator`를 검증하기 위해 다음 예시를 살펴봅시다. 다시 한 번 약의 효과를 찾아봅시다. 약의 효과는 성별(sex), 나이(age), 중증도(severity)에 영향을 받습니다. 또한 중증환자는 약을 받을 가능성이 더 높습니다.

In [11]:
med = pd.read_csv("./data/medicine_impact_recovery.csv")
med.head()

,sex,age,severity,medication,recovery
0,0,35.049134,0.887658,1,31
1,1,41.580323,0.899784,1,49
2,1,28.127491,0.486349,0,38
3,1,36.375033,0.323091,0,35
4,0,25.091717,0.209006,0,15


단순한 평균 차이인 $E[Y|T=1]-E[Y|T=0]$에 따르면 약을 처방할 때 회복에 16.9일 더 걸립니다. 하지만 약이 회복에 도움이 될 것으로 생각되므로 `confounder`에 의한 편향일 것입니다.

In [12]:
med.query("medication==1")["recovery"].mean() - med.query("medication==0")["recovery"].mean()

16.895799546498726

편향을 없애기 위해 매칭을 사용해 `X`를 통제합니다. 먼저, `feature`의 크기를 맞춰줍니다. 크기가 다르면 중요도가 달리 계산되기 때문입니다. 평균과 표준편차로 표준화합니다.

In [13]:
# scale features
X = ["severity", "age", "sex"]
y = "recovery"

med = med.assign(**{f: (med[f] - med[f].mean())/med[f].std() for f in X})
med.head()

,sex,age,severity,medication,recovery
0,-0.996980,0.280787,1.459800,1,31
1,1.002979,0.865375,1.502164,1,49
2,1.002979,-0.338749,0.057796,0,38
3,1.002979,0.399465,-0.512557,0,35
4,-0.996980,-0.610473,-0.911125,0,15


이제 매칭해봅시다. 매칭 함수를 만드는 대신 [Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)의 `K nearest neighbour algorithm`(KNN)을 사용합니다. KNN은 훈련 및 검증 데이터 세트에서 가장 유사한 데이터를 찾아줍니다.

여기서는 2개 모델이 필요합니다. `mt0`는 `untreated` 데이터로 훈련되며, `treated` 데이터를 `untreated` 데이터와 매칭해 줍니다. `mt1`는 `treated` 데이터로 훈련되며, `untreated` 데이터를 `treated` 데이터와 매칭해 줍니다. KNN 훈련을 마치고 훈련된 모델로 예측을 수행합니다.

In [14]:
from sklearn.neighbors import KNeighborsRegressor

treated = med.query("medication==1")
untreated = med.query("medication==0")

mt0 = KNeighborsRegressor(n_neighbors=1).fit(untreated[X], untreated[y])
mt1 = KNeighborsRegressor(n_neighbors=1).fit(treated[X], treated[y])

predicted = pd.concat([
    # find matches for the treated looking at the untreated knn model
    treated.assign(match=mt0.predict(treated[X])),
    
    # find matches for the untreated looking at the treated knn model
    untreated.assign(match=mt1.predict(untreated[X]))
])

predicted.head()

,sex,age,severity,medication,recovery,match
0,-0.996980,0.280787,1.459800,1,31,39.0
1,1.002979,0.865375,1.502164,1,49,52.0
7,-0.996980,1.495134,1.268540,1,38,46.0
10,1.002979,-0.106534,0.545911,1,34,45.0
16,-0.996980,0.043034,1.428732,1,30,39.0


이제 `matching estimator` 공식을 적용합니다.

$
\hat{ATE} = \frac{1}{N} \sum^N_{i=0} (2T_i - 1)\big(Y_i - Y_{jm}(i)\big)
$

In [15]:
np.mean((2*predicted["medication"] - 1)*(predicted["recovery"] - predicted["match"]))

-0.9954

약의 효과는 더는 양수가 아닙니다. `X`를 통제할 때 약은 평균적으로 회복 시간을 약 하루 줄여줍니다. 회복 시간이 16.9시간 길어질 것으로 예측한 편향된 추정치에 비하면 큰 개선입니다.

하지만 더 잘할 수 있습니다.

## Matching Bias

위의 `matching estimator`는 편향되었습니다. 확인을 위해 `ATE` 대신 `ATET` 추정기를 생각해 봅시다. 여기서 얻은 직관을 `ATE`에도 적용할 수 있습니다.

$
\hat{ATET} = \frac{1}{N_1}\sum (Y_i - Y_j(i))
$

$N_1$은 `treatment`가 적용된 데이터 개수이고 $Y_j(i)$는 `treated` 데이터인 unit i를 이용하여 매칭을 통해 얻은 $i$번째 unit이 untreated 되었다면 가졌을 `outcome`입니다. 얻어진 \hat{ATET}이 편향이 있는지 확인하기 위해 중심극한정리를 적용하여 아래 식이 평균이 0인 정규분포에 수렴하는지 확인할 것입니다. 

$
\sqrt{N_1}(\hat{ATET} - ATET)
$

하지만 항상 0이 되지는 않습니다. 주어진 `X`에 대해 `untreated`의 `outcome` 평균을  $\mu_0(x)=E[Y|X=x, T=0]$ 로 정의하면, 아래식과 같이 바뀝니다. (여기서는 증명을 생략합니다.)

$
E[\sqrt{N_1}(\hat{ATET} - ATET)] = E[\sqrt{N_1}(\mu_0(X_i) - \mu_0(X_j(i)))]
$

$\mu_0(X_i) - \mu_0(X_j(i))$는 이해가 어려울 수 있어 자세히 살펴보겠습니다. $\mu_0(X_i)$는 처치된 unit $i$가 처치가 되지 않았다면 가졌을 `outcome` `Y`입니다. 즉, `unit` $i$에 대한 `counterfactual outcome` $Y_0$입니다. $\mu_0(X_j(i))$는 `unit` $i$와 매칭된 `unit` $j$의 `untreated outcome`입니다. 똑같이 $Y_0$이지만 $j$번째 데이터 입니다. $j$는 처치되지 않은 그룹에 속하므로 `factual outcome`입니다. $i$와 $j$는 비슷하긴 하지만 같지는 않으므로 0이 아닐 수 있습니다. 즉, $X_i \approx X_j $입니다. 따라서 $Y_{0i} \approx Y_{0j} $입니다.

데이터가 많으면 매칭할 `unit`이 많아지므로 $i$와 $j$간 차이도 작아질 것입니다. 하지만 차이가 0으로 수렴하는 속도는 늦습니다. 결국 $\mu_0(X_i) - \mu_0(X_j(i))$ 보다 $\sqrt{N_1}$가 빠르게 커지므로 $E[\sqrt{N_1}(\mu_0(X_i) - \mu_0(X_j(i)))]$는 0이 아닐 수 있습니다.

편향은 매칭 `unit`간 차이가 클 때 발생합니다. 문제를 해결할 방법은 있습니다. 각 관측값이 편향에 기여하는 양은 $(\mu_0(X_i) - \mu_0(X_j(i)))$ 이므로, matching estimator 내에서 각각의 매칭 비교항마다 이 양을 빼면 됩니다. 이를 위해 $\mu_0(X_j(i))$를 선형회귀 같은 모델로 얻을 수 있는 $\hat{\mu_0}(X_j(i))$로 대체할 수 있습니다. `ATET` 추정기는 다음 방정식으로 업데이트할 수 있습니다.

$
\hat{ATET} = \frac{1}{N_1}\sum \big((Y_i - Y_{j(i)}) - (\hat{\mu_0}(X_i) - \hat{\mu_0}(X_{j(i)}))\big)
$

$\hat{\mu_0}(x)$는 `untreated` 샘플로만 훈련된 선형회귀 같은 모델의 $E[Y|X, T=0]$ 추정치입니다.

In [16]:
from sklearn.linear_model import LinearRegression

# fit the linear regression model to estimate mu_0(x)
ols0 = LinearRegression().fit(untreated[X], untreated[y])
ols1 = LinearRegression().fit(treated[X], treated[y])

# find the units that match to the treated
treated_match_index = mt0.kneighbors(treated[X], n_neighbors=1)[1].ravel()

# find the units that match to the untreatd
untreated_match_index = mt1.kneighbors(untreated[X], n_neighbors=1)[1].ravel()

predicted = pd.concat([
    (treated
     # find the Y match on the other group
     .assign(match=mt0.predict(treated[X])) 
     
     # build the bias correction term
     .assign(bias_correct=ols0.predict(treated[X]) - ols0.predict(untreated.iloc[treated_match_index][X]))),
    (untreated
     .assign(match=mt1.predict(untreated[X]))
     .assign(bias_correct=ols1.predict(untreated[X]) - ols1.predict(treated.iloc[untreated_match_index][X])))
])

predicted.head()

,sex,age,severity,medication,recovery,match,bias_correct
0,-0.996980,0.280787,1.459800,1,31,39.0,4.404034
1,1.002979,0.865375,1.502164,1,49,52.0,12.915348
7,-0.996980,1.495134,1.268540,1,38,46.0,1.871428
10,1.002979,-0.106534,0.545911,1,34,45.0,-0.496970
16,-0.996980,0.043034,1.428732,1,30,39.0,2.610159


한 가지 질문은 다음과 같습니다. 이 방법이 매칭을 쓸모없게 만들지는 않을까요? 어쨌든 선형회귀를 사용한다면 복잡한 모델 대신에 선형회귀만 사용하는 것이 어떨까요? 일리가 있는 지적입니다. 시간을 두고 답변드리겠습니다.

![img](./data/img/matching/ubiquitous-ols.png)

우선 앞에서 사용한 선형회귀는 `treatment`에 대한 외삽을 하지 않습니다. 선형회귀의 목적은 편향을 줄이는 것입니다. 선형회귀는 국소적입니다. `treated`와 `untreated`를 비교하려 하지 않습니다. 어떠한 외삽도 하지 않아요. `treatment effect`는 매칭 알고리즘이 맡습니다. 주장하려는 바는 `OLS`가 부가적이라는 것입니다.

두 번째 요점은 매칭이 비모수 추정기라는 것입니다. 선형성이나 모수를 가정하지 않습니다. 따라서 선형회귀 분석보다 유연하며 비선형성이 매우 강한 상황에서도 사용할 수 있습니다.

매칭만 사용하라는 의미인가요? 어려운 질문입니다. Alberto Abadie는 그래야 한다고 주장합니다. 더 유연하고 간단히 수행할 수 있습니다. (코드만 있다면요.) 하지만 저는 전적으로 동의하지 않습니다. Abadie는 매칭을 연구하고 개발하는 데 많은 시간을 보냈기 때문에 (그는 매칭 알고리즘을 개발하는데 기여한 과학자 중 한 명입니다.) 분명히 개인적인 취향이 반영되어 있습니다. 또한, 매칭 없는 선형회귀의 단순함은 장점입니다. "모든 것을 일정하게"해야 하는 편미분과 관련된 문제는 선형회귀로 파악하기가 보다 쉽습니다. 하지만 제 의견일 뿐이에요. 솔직히 이 질문에 대한 명확한 답은 없습니다. 어쨌든, 예시로 돌아갑시다.

편향 보정 공식으로 아래와 같은 `ATE`를 얻습니다.

In [17]:
np.mean((2*predicted["medication"] - 1)*((predicted["recovery"] - predicted["match"])-predicted["bias_correct"]))

-7.362660906141416

물론 신뢰 구간을 계산해야 하지만 수학적 이론은 이것으로 충분합니다. 간편하게 다른 사람의 코드를 사용해 추정치를 얻을 수 있습니다. 다음은 [causalinference](https://github.com/laurencium/causalinference)을 사용한 결과입니다.

In [18]:
from causalinference import CausalModel

cm = CausalModel(
    Y=med["recovery"].values, 
    D=med["medication"].values, 
    X=med[["severity", "age", "sex"]].values
)

cm.est_via_matching(matches=1, bias_adj=True)

print(cm.estimates)


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -7.709      0.609    -12.649      0.000     -8.903     -6.514
           ATC     -6.665      0.246    -27.047      0.000     -7.148     -6.182
           ATT     -9.679      1.693     -5.717      0.000    -12.997     -6.361



이제 약이 입원 기간을 실제로 줄여준다고 자신 있게 말할 수 있습니다. `ATE` 추정치는 제가 직접 작성한 코드의 결과보다 조금 낮기 때문에 제 코드가 완벽하지 않을 수도 있습니다. 잘 만들어진 코드를 가져와야 하는 이유입니다.

마지막으로 매칭 편향의 원인을 다루겠습니다. 매칭 결과가 비슷하지 않을 때 편향된다는 것은 알고 있습니다. 무엇이 매칭 결과를 다르게 만들까요?

## The Curse of Dimensionality

정답은 꽤 단순하고 직관적입니다. `sex`와 같이 하나의 특성에 대해서는 일치하는 데이터를 쉽게 찾을 수 있습니다. 하지만 나이, 소득, 출신지와 같은 `feature`를 더할수록 비슷한 데이터는 점점 찾기 어려워집니다. 일반적으로 `feature`가 많아지면 매칭 결과의 거리는 멀어집니다.

`matching estimator` 뿐만 아니라 `subclassification estimator`에도 같은 문제가 있습니다. 처음에 약 처방 사례에서 `subclassification estimator`를 만들기 쉬웠던 이유는 남성, 여성 두 개 값만 있었기 때문입니다. 하지만 많은 변수가 있었다면 어떨까요? 나이나 소득과 같이 연속적인 변수가 있고 각각 5개 범위로 분리한다고 해봅시다. $5^2=25$개의 조합을 고려해야 합니다. 10개 변수가 각각 3개 범위로 나뉜다면 어떨까요? 이제 $3^{10}=59049$개 조합을 다뤄야 합니다. 아주 빠르게 데이터가 희소해짐을 알 수 있습니다. **차원의 저주**라고 불리는 모든 데이터 과학에서 관찰되는 현상입니다!

![img](./data/img/curse-of-dimensionality.jpg)

Image Source: https://deepai.org/machine-learning-glossary-and-terms/curse-of-dimensionality

무서운 이름이지만 `feature space`를 채우는 데 필요한 데이터 수가 기하급수적으로 증가한다는 것을 의미할 뿐입니다. 4차원 `feature space`를 채우는 데는 3차원의 경우보다 훨씬 많은 데이터가 필요합니다.

`subclassification estimator`의 경우 차원의 저주는 `feature`가 많을수록 일을 어렵게 만듭니다. 데이터값의 여럿이라면 일치하는 데이터는 거의 없습니다. 극히 일부의 실험군과 대조군을 얻기 때문에 `ATE`를 추정하는 것은 거의 불가능합니다. 매칭을 하더라도 `unit`의 거리가 매우 멀게 되고 결국 편향이 발생합니다.

선형회귀는 이 문제를 상당히 잘 해결합니다. 모든 `feature` X를 단일 차원으로 만들어 실험군과 대조군을 비교합니다. 우아한 방법으로 일종의 차원 축소를 수행합니다.  

대부분의 인과모델은 차원의 저주를 다루는 방법을 가지고 있습니다. 계속 반복하지는 않겠지만, 명심 해야 합니다. 다음 장에서 성향 점수를 다루면서 문제를 어떻게 해결하는지 공부해봅시다.

## Key Ideas

10장은 인과관계를 구할 때 선형회귀의 메커니즘을 이해하는 것으로 시작했습니다. 선형회귀는 데이터를 값으로 분할하여 `ATE`를 계산한 다음 전체 데이터에 대해 단일 `ATE`로 결합하는 것으로 볼 수 있습니다.

`subclassification estimator`를 공부했습니다. 실제로 유용하지는 않지만 인과추론에 대한 직관을 주었습니다. 또한, `subclassification estimator`를 `matching estimator`로 확장할 수 있었습니다.

매칭은 `confounder`를 통제하여 실험군과 대조군 쌍을 만듭니다. KNN 알고리즘으로 매칭을 구현했으며 선형회귀로 편향을 줄이는 방법도 공부했습니다. 또한, 매칭과 선형회귀의 차이를 살펴보았습니다. 매칭은 비모수 추정기로 선형회귀와 다르게 선형성에 의존하지 않습니다.

마지막으로 고차원 데이터 문제를 다뤘으며 차원의 저주가 인과추론을 왜 어렵게 만드는지 살펴보았습니다.

## References

I like to think of this entire book as a tribute to Joshua Angrist, Alberto Abadie and Christopher Walters for their amazing Econometrics class. Most of the ideas here are taken from their classes at the American Economic Association. Watching them is what is keeping me sane during this tough year of 2020.
* [Cross-Section Econometrics](https://www.aeaweb.org/conference/cont-ed/2017-webcasts)
* [Mastering Mostly Harmless Econometrics](https://www.aeaweb.org/conference/cont-ed/2020-webcasts)

I'll also like to reference the amazing books from Angrist. They have shown me that Econometrics, or 'Metrics as they call it, is not only extremely useful but also profoundly fun.

* [Mostly Harmless Econometrics](https://www.mostlyharmlesseconometrics.com/)
* [Mastering 'Metrics](https://www.masteringmetrics.com/)

My final reference is Miguel Hernan and Jamie Robins' book. It has been my trustworthy companion in the most thorny causal questions I had to answer.

* [Causal Inference Book](https://www.hsph.harvard.edu/miguel-hernan/causal-inference-book/)

![img](./data/img/poetry.png)

## Contribute

Causal Inference for the Brave and True is an open-source material on causal inference, the statistics of science. It uses only free software, based in Python. Its goal is to be accessible monetarily and intellectually.
If you found this book valuable and you want to support it, please go to [Patreon](https://www.patreon.com/causal_inference_for_the_brave_and_true). If you are not ready to contribute financially, you can also help by fixing typos, suggesting edits or giving feedback on passages you didn't understand. Just go to the book's repository and [open an issue](https://github.com/matheusfacure/python-causality-handbook/issues). Finally, if you liked this content, please share it with others who might find it useful and give it a [star on GitHub](https://github.com/matheusfacure/python-causality-handbook/stargazers).